# **Goal: Recommendation System / Next Best Action**

## **Context**
We have a **random extraction** of a (real world) dataset containing **customers of a large wealth management company**.  

The data is **anonymous, mostly clean, and NOT always normalized/scaled**.  

Our objective is to **estimate investment needs** for these customers using **Data Science techniques**.

### **Why Estimate Investment Needs?**
Identifying customer needs is useful for several reasons, including:

* **Recommender Systems / Next Best Action:**  
  * Needs can serve as **key inputs** for **content-based** or **knowledge-based filtering algorithms**, that allows for personalized services.  
  * This is our **primary focus** in this notebook, i.e., "Know Your Client (KYC)".  

* **Product Targeting & Governance (Regulatory Compliance - MIFID/IDD in EU):**  
  * Regulatory standards require that **customer needs match the investment products offered**. So financial institutions must estimate customer needs.
  * This is essentially an **"institutional view"** of a recommendation system...

<br>

---

## **Dataset Overview**
The dataset, named **"Needs"**, is stored in an **Excel file called Dataset2_Needs.xls**.  
It contains several **potentially relevant features** along with two **target variables**, i.e:

* **AccumulationInvestment**  
  * Indicates a customer preference for **accumulation investing**, typically through **dollar-cost averaging** (i.e., investing small amounts at regular intervals over time, say on a monthly basis).  
  * **Binary (Boolean) response:**  
    * `1 = High propensity`.  
    * `0 = Low propensity`.

- **IncomeInvestment**  
  - Indicates a customer preference for **income investing**, typically through **lump-sum investing** (i.e., one-shot investments).  This is because anyone who aspires to obtain income from coupons and dividends must necessarily already have accumulated capital - a typical need of people who are older than their previous need.
  - **Binary (Boolean) response:**  
    - `1 = High propensity`.  
    - `0 = Low propensity`.  

    **Where do these two response variables come from?** From a **revealed preference scheme**: if the client has an advisor who is considered professionally reliable (this eliminates the possibility of conflict of interest) and has purchased a product that satisfies that need, and the client has also purchased it, we can say with good probability that the advisor has identified the need correctly and the client has that need. In other respects, the machine learning model we are building is a clone of the financial advisor.

<br>

Additionally, we have a **second dataset**, **"Products"**, containing investment products (funds, segregated accounts, unit-linked policies), along with:

* **Product Type:**  
  * `1 = Accumulation` (that is, a product that is good for those who have a high need for accumulation investments)
  * `0 = Income`  (that is, a product that is good for those who have a high need for income investments)

* **Risk Level:**  
  * A **normalized risk score** in the range **$[0,1]$**.  
  * This usually represents the normalized value in $[0, 1]$ of the **[Synthetic Risk and Reward Indicator (SRRI)](https://www.esma.europa.eu/sites/default/files/library/2015/11/10_673.pdf)** of the product, an ordinal variable defined in the range ${1, 7}$ starting from continuous data.

<br>

---

## **Recommendation System Approach**
The recommendation system consists of **two key steps**:

1. **Identifying customers with high investment propensity:**  
   - Using **machine learning models**, we aim to classify customers based on **AccumulationInvestment** (`1 = High propensity`) and/or **IncomeInvestment** (`1 = High propensity`).  

2. **Recommending the most suitable product for each customer:**  
   - For each customer, we match the **most appropriate product** based on:  
     - **Investment need** (Accumulation or Income).  
     - **Risk compatibility** (matching product risk level with the customer profile).  
   - This **personalized recommendation** represents the **Next Best Action** for each client.

<br>


<br>

Let's start with data ingestion.

<br>

In [1]:
import pandas as pd

In [2]:
import os

folder = r'C:\Users\leo_h\Documents\project\fintech_final_project'
print(os.listdir(folder))

['.git', '.vscode', 'BaseNotebook.ipynb', 'dataset', 'Dataset2_Needs.xls', 'Dataset2_Needs.xlsx', 'FeaEngwithDecisionTree.ipynb', 'Projectwork_Zenti.pdf', 'Zenti_Business_Case_2.pdf']


In [4]:
import os

folder = r'C:\Users\leo_h\Documents\project\fintech_final_project'
file_path = "Dataset2_Needs.xlsx"

os.chdir(folder)
os.getcwd()
# Print current working directory for debugging
print("Current working directory:", os.getcwd())
print("Looking for file at:", file_path)

# 'folder' and 'file_path' are already defined in the notebook, so we use them directly



needs_df = pd.read_excel(file_path, sheet_name='Needs')
products_df = pd.read_excel(file_path, sheet_name='Products')
metadata_df = pd.read_excel(file_path, sheet_name='Metadata')

Current working directory: C:\Users\leo_h\Documents\project\fintech_final_project
Looking for file at: Dataset2_Needs.xlsx


# **Data Exploration**

As for the last business case: I keep it minimalist, for the benefit of brevity, to be able to get to the heart of the problem. But you could/can spend tons of time here in order to **understand the problem and the dataset**.

Let's display our variables to better understand the data structure and characteristics of the dataset.

<br>

In [5]:
# Let's see the actual variables names in metadata_df
print("Metadata DataFrame columns:")
print(metadata_df.columns.tolist())

# Let's peek at the first few rows
print("\nFirst few rows of metadata:")
print(metadata_df.head())


Metadata DataFrame columns:
['Metadata', 'Unnamed: 1']

First few rows of metadata:
        Metadata                     Unnamed: 1
0        Clients                            NaN
1             ID                   Numerical ID
2            Age                  Age, in years
3         Gender  Gender (Female = 1, Male = 0)
4  FamilyMembers           Number of components


<br>

We drop ID column as it's not needed for analysis.

<br>

In [6]:
# Drop ID column as it's not needed for analysis
needs_df = needs_df.drop('ID', axis=1)

<br>

Create a formatted table to summarize the dataset (you can expand the number of statistics you look at).


<br>

In [7]:
def create_variable_summary(df, metadata_df):
    # Create empty lists to store the chosen statistics
    stats_dict = {
        'Variable': [],
        'Description': [],
        'Mean': [],
        'Std': [],
        'Missing': [],
        'Min': [],
        'Max': []
    }

    # Create a metadata dictionary for easy lookup
    meta_dict = dict(zip(metadata_df['Metadata'], metadata_df['Unnamed: 1']))

    for col in df.columns:
        stats_dict['Variable'].append(col)
        stats_dict['Description'].append(meta_dict.get(col, 'N/A'))

        # Calculate some statistics for each column
        if pd.api.types.is_numeric_dtype(df[col]):
            stats_dict['Mean'].append(f"{df[col].mean():.2f}")
            stats_dict['Std'].append(f"{df[col].std():.2f}")
            stats_dict['Min'].append(f"{df[col].min():.2f}")
            stats_dict['Max'].append(f"{df[col].max():.2f}")
        else:
            stats_dict['Mean'].append('N/A')
            stats_dict['Std'].append('N/A')
            stats_dict['Min'].append('N/A')
            stats_dict['Max'].append('N/A')

        stats_dict['Missing'].append(df[col].isna().sum())

    return pd.DataFrame(stats_dict)


# Create summary tables
print("NEEDS VARIABLES SUMMARY:")
needs_summary = create_variable_summary(needs_df, metadata_df)
display(needs_summary.style
        .set_properties(**{'text-align': 'left'})
        .hide(axis='index'))

print("\nPRODUCTS VARIABLES SUMMARY:")
products_summary = create_variable_summary(products_df, metadata_df)
display(products_summary.style
        .set_properties(**{'text-align': 'left'})
        .hide(axis='index'))


NEEDS VARIABLES SUMMARY:


Variable,Description,Mean,Std,Missing,Min,Max
Age,"Age, in years",55.25,11.97,0,18.00,97.00
Gender,"Gender (Female = 1, Male = 0)",0.49,0.50,0,0.00,1.00
FamilyMembers,Number of components,2.51,0.76,0,1.00,5.00
FinancialEducation,Normalized level of Financial Education (estimate),0.42,0.15,0,0.04,0.90
RiskPropensity,Normalized Risk propensity from MIFID profile,0.36,0.15,0,0.02,0.88
Income,Income (thousands of euros); estimate,62.99,44.36,0,1.54,365.32
Wealth,Wealth (thousands of euros); sum of investments and cash accounts,93.81,105.47,0,1.06,2233.23
IncomeInvestment,Boolean variable for Income investment; 1 = High propensity,0.38,0.49,0,0.00,1.00
AccumulationInvestment,Boolean variable for Accumulation/growth investment; 1 = High propensity,0.51,0.50,0,0.00,1.00



PRODUCTS VARIABLES SUMMARY:


Variable,Description,Mean,Std,Missing,Min,Max
IDProduct,Product description,6.00,3.32,0,1.00,11.00
Type,"1 = Accumulation product, 0 = Income product",0.64,0.50,0,0.00,1.00
Risk,Normalized Synthetic Risk Indicator,0.43,0.24,0,0.12,0.88


<h1> Feature Engineering

<body>
<h2>Feature Idea 8: Gender × Age Interaction</h2>
Rationale: Investigate whether life-cycle financial needs differ by gender within the dataset. Additionally, refine the Income/Wealth Ratio by applying a logarithmic transformation for better scaling.
Combining the Gender × Age interaction feature with the log-transformed Income/Wealth Ratio and the base feature set yielded the strongest performance among all feature engineering strategies tested.
</body>

In [ ]:
import pandas as pd
import numpy as np

# --- Sklearn Imports ---
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler
# --- Add imports for new models ---

from sklearn.tree import DecisionTreeClassifier

# ---------------------------------
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tabulate import tabulate

# --- Global Settings ---
RANDOM_STATE = 42

# --- ASSUME 'needs_df' is loaded and preprocessed correctly before this point ---
# Example:
# needs_df = pd.read_csv('your_data.csv')
# # Ensure Gender, Age, FamilyMembers are numeric
# needs_df['Gender'] = needs_df['Gender'].apply(lambda x: 1 if x == 'Male' else 0)
# needs_df['FamilyMembers'] = pd.to_numeric(needs_df['FamilyMembers'], errors='coerce').fillna(0)
# needs_df['Age'] = pd.to_numeric(needs_df['Age'], errors='coerce').fillna(needs_df['Age'].median())


# Step 1: Feature engineering function (Implementing Both New Features)
def prepare_features(df):
    """Prepares base and engineered (Gender*Age + I/W Ratio) feature sets."""
    X = df.copy()
    income_col = 'Income ' if 'Income ' in X.columns else 'Income'
    wealth_col = 'Wealth'
    gender_col = 'Gender'
    age_col = 'Age'

    # Log transformation for Wealth and Income
    if wealth_col in X.columns: X['Wealth_log'] = np.log1p(X[wealth_col])
    if income_col in X.columns: X['Income_log'] = np.log1p(X[income_col])

    # --- Engineered Feature 1: Gender * Age Interaction ---
    interaction_gender_age = 'Gender_x_Age'
    if age_col in X.columns and gender_col in X.columns and \
       pd.api.types.is_numeric_dtype(X[age_col]) and \
       pd.api.types.is_numeric_dtype(X[gender_col]):
        X[interaction_gender_age] = X[age_col] * X[gender_col]
    else:
        X[interaction_gender_age] = 0
        print(f"Warning: Could not calculate {interaction_gender_age}.")

    # --- Engineered Feature 2: Refined Income/Wealth Ratio (log) ---
    feature_iw_ratio_log = 'Income_Wealth_Ratio_log'
    if income_col in X.columns and wealth_col in X.columns and \
       pd.api.types.is_numeric_dtype(X[income_col]) and \
       pd.api.types.is_numeric_dtype(X[wealth_col]):
        # Calculate ratio using original values, handle division by zero
        ratio = X[income_col].div(X[wealth_col].replace(0, np.nan))
        # Fill NaN ratios with 0 before log1p
        X[feature_iw_ratio_log] = np.log1p(ratio.fillna(0))
    else:
        X[feature_iw_ratio_log] = 0
        print(f"Warning: Could not calculate {feature_iw_ratio_log}.")


    # --- Define Feature Lists ---
    # Base features: Standard set
    features_base_expected = ['Age', 'Gender', 'FamilyMembers', 'FinancialEducation',
                              'RiskPropensity', 'Wealth_log', 'Income_log']

    # Engineered features: Base + BOTH New Features
    features_engineered_expected = features_base_expected.copy()
    if interaction_gender_age in X.columns:
         features_engineered_expected.append(interaction_gender_age)
    if feature_iw_ratio_log in X.columns:
         features_engineered_expected.append(feature_iw_ratio_log)

    # Select only available columns
    features_base = [f for f in features_base_expected if f in X.columns]
    features_engineered = [f for f in features_engineered_expected if f in X.columns]


    # Normalize all features
    scaler_base = MinMaxScaler()
    scaler_eng = MinMaxScaler()

    # Minimal error checking assumed data is clean/numeric
    X_base = pd.DataFrame(scaler_base.fit_transform(X[features_base]), columns=features_base, index=X.index)
    X_engineered = pd.DataFrame(scaler_eng.fit_transform(X[features_engineered]), columns=features_engineered, index=X.index)

    return X_base, X_engineered


In [104]:
X_base, X_engineered = prepare_features(needs_df)
y_income = needs_df['IncomeInvestment']
y_accum = needs_df['AccumulationInvestment']

In [ ]:
# Base Features
print("Base Features:")
X_base.head(5)

Base Features:


,Age,Gender,FamilyMembers,FinancialEducation,RiskPropensity,Wealth_log,Income_log
0,0.531646,0.0,0.25,0.222172,0.243105,0.468132,0.664782
1,0.759494,0.0,0.25,0.372410,0.170321,0.600160,0.441614
2,0.189873,1.0,0.25,0.324649,0.262161,0.498951,0.453970
3,0.645570,1.0,0.75,0.843975,0.734110,0.756044,0.842246
4,0.506329,0.0,0.50,0.454090,0.377948,0.482307,0.436064


In [105]:

# Engineered Features
print("\nEngineered Features:")
X_engineered.head(5)


Engineered Features:


,Age,Gender,FamilyMembers,FinancialEducation,RiskPropensity,Wealth_log,Income_log,Gender_x_Age,Income_Wealth_Ratio_log
0,0.531646,0.0,0.25,0.222172,0.243105,0.468132,0.664782,0.000000,0.155428
1,0.759494,0.0,0.25,0.372410,0.170321,0.600160,0.441614,0.000000,0.025499
2,0.189873,1.0,0.25,0.324649,0.262161,0.498951,0.453970,0.347368,0.054647
3,0.645570,1.0,0.75,0.843975,0.734110,0.756044,0.842246,0.726316,0.062964
4,0.506329,0.0,0.50,0.454090,0.377948,0.482307,0.436064,0.000000,0.055916


In [106]:
# Income Investment Model
y_income


0       0
1       1
2       0
3       1
4       0
       ..
4995    0
4996    1
4997    0
4998    0
4999    1
Name: IncomeInvestment, Length: 5000, dtype: int64

In [107]:
# Accumulation Investment Model
y_accum

0       1
1       0
2       1
3       1
4       0
       ..
4995    0
4996    1
4997    1
4998    0
4999    1
Name: AccumulationInvestment, Length: 5000, dtype: int64

<h2>Model Training and Evaluation</h2>

<h3>DecisionTreeClassifier</h3>

In [108]:

# Step 2: Data split function (Professor's Original)
def split_data(X, y, test_size=0.2, random_state=RANDOM_STATE):
    """Original split function."""
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )
    return X_train, X_test, y_train, y_test

# Step 3: Model training and evaluation function (Professor's Original + zero_division)
def train_evaluate_model(X_train, y_train, X_test, y_test, model, k_folds=5):
    """Original evaluation function with robust metric calculation."""
    kf = KFold(n_splits=k_folds, shuffle=True, random_state=RANDOM_STATE)
    cv_metrics = {'accuracy': [], 'precision': [], 'recall': [], 'f1': []}

    for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
        X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]
        try:
            model.fit(X_train_fold, y_train_fold)
            y_val_pred = model.predict(X_val_fold)
            cv_metrics['accuracy'].append(accuracy_score(y_val_fold, y_val_pred))
            cv_metrics['precision'].append(precision_score(y_val_fold, y_val_pred, zero_division=0))
            cv_metrics['recall'].append(recall_score(y_val_fold, y_val_pred, zero_division=0))
            cv_metrics['f1'].append(f1_score(y_val_fold, y_val_pred, zero_division=0))
        except Exception as e:
            print(f"Error in CV fold {fold} for {model.__class__.__name__}: {e}")
            for metric in cv_metrics: cv_metrics[metric].append(0)

    test_metrics_calculated = {m: 0 for m in ['accuracy', 'precision', 'recall', 'f1']}
    try:
        model.fit(X_train, y_train)
        y_test_pred = model.predict(X_test)
        test_metrics_calculated = {
            'accuracy': accuracy_score(y_test, y_test_pred),
            'precision': precision_score(y_test, y_test_pred, zero_division=0),
            'recall': recall_score(y_test, y_test_pred, zero_division=0),
            'f1': f1_score(y_test, y_test_pred, zero_division=0) }
    except Exception as e:
        print(f"Error during final fit/predict for {model.__class__.__name__}: {e}")

    return { 'cv_metrics': { metric: {'mean': np.mean(scores), 'std': np.std(scores)}
                           for metric, scores in cv_metrics.items() },
             'test_metrics': test_metrics_calculated }

# Step 4: Display results function (Professor's Original - default index)
def display_results_table(results_dict, model_name, feature_type):
    """Original display function showing default 0,1,2,3 index."""
    cv_data = {
        'Metric': ['Accuracy', 'Precision', 'Recall', 'F1'],
        'CV Mean': [ results_dict['cv_metrics']['accuracy']['mean'], results_dict['cv_metrics']['precision']['mean'],
                     results_dict['cv_metrics']['recall']['mean'], results_dict['cv_metrics']['f1']['mean'] ],
        'CV Std': [ results_dict['cv_metrics']['accuracy']['std'], results_dict['cv_metrics']['precision']['std'],
                    results_dict['cv_metrics']['recall']['std'], results_dict['cv_metrics']['f1']['std'] ],
        'Test Set': [ results_dict['test_metrics']['accuracy'], results_dict['test_metrics']['precision'],
                      results_dict['test_metrics']['recall'], results_dict['test_metrics']['f1'] ]
    }
    df = pd.DataFrame(cv_data).round(3)
    print(f"\n{model_name} - {feature_type}")
    print("=" * 60)
    print(tabulate(df, headers='keys', tablefmt='pretty', showindex="always")) # Show default 0,1,2,3 index



In [109]:
# Step 5: Run full analysis (Original Structure + Expanded Models)

# --- Prepare features ---
# Assuming needs_df exists and is preprocessed


# --- Define models (Expanded Dictionary) ---
models = {
   
    'DecisionTree': DecisionTreeClassifier(max_depth=5, random_state=RANDOM_STATE),
  
}
# -----------------------------------------

# --- Actually run analysis and display results ---
for target_name, y in [('Income Investment', y_income), ('Accumulation Investment', y_accum)]:
    print(f"\nTarget Variable: {target_name}")
    print("=" * 80)

    X_base_train, X_base_test, y_train_base, y_test_base = split_data(X_base, y)
    X_eng_train, X_eng_test, y_train_eng, y_test_eng = split_data(X_engineered, y)
    y_train = y_train_base ; y_test = y_test_base # Use common y split

    # --- Evaluate Models ---
    for model_name, model_prototype in models.items():
        print(f"\n--- Evaluating {model_name} ---")
        model_for_base = model_prototype.__class__(**model_prototype.get_params())
        model_for_eng = model_prototype.__class__(**model_prototype.get_params())

        results_base = train_evaluate_model(X_base_train, y_train, X_base_test, y_test, model_for_base)
        display_results_table(results_base, model_name, "Base Features")

        # *** Update label for this engineered set ***
        engineered_set_name = "Eng Features (Gender*Age + I/W Ratio)" # More descriptive label
        results_eng = train_evaluate_model(X_eng_train, y_train, X_eng_test, y_test, model_for_eng)
        display_results_table(results_eng, model_name, engineered_set_name)

print("\nAnalysis finished.")


Target Variable: Income Investment

--- Evaluating DecisionTree ---

DecisionTree - Base Features
+---+-----------+---------+--------+----------+
|   |  Metric   | CV Mean | CV Std | Test Set |
+---+-----------+---------+--------+----------+
| 0 | Accuracy  |  0.795  | 0.006  |  0.767   |
| 1 | Precision |  0.829  | 0.026  |  0.794   |
| 2 |  Recall   |  0.588  | 0.024  |  0.531   |
| 3 |    F1     |  0.688  | 0.012  |  0.637   |
+---+-----------+---------+--------+----------+

DecisionTree - Eng Features (Gender*Age + I/W Ratio)
+---+-----------+---------+--------+----------+
|   |  Metric   | CV Mean | CV Std | Test Set |
+---+-----------+---------+--------+----------+
| 0 | Accuracy  |  0.824  | 0.009  |  0.789   |
| 1 | Precision |  0.887  | 0.021  |  0.859   |
| 2 |  Recall   |  0.62   | 0.032  |  0.539   |
| 3 |    F1     |  0.729  | 0.018  |  0.662   |
+---+-----------+---------+--------+----------+

Target Variable: Accumulation Investment

--- Evaluating DecisionTree ---

Dec

<body>
# Results Analysis: Base Features vs. Engineered Features (Gender × Age + I/W Ratio)

## Decision Tree Performance
- **Income Investment:**  
  - Base F1 = 0.637  
  - Engineered F1 = 0.662  
  - *Engineered features provided a clear improvement.*

- **Accumulation Investment:**  
  - Base F1 = 0.664  
  - Engineered F1 = 0.818  
  - *Engineered features led to a dramatic performance increase.*

## Model Insights
- For **Income Investment**, engineered features notably improved the Decision Tree and provided modest gains for SVM and Naive Bayes, but slightly reduced performance for Logistic Regression and KNN. Despite mixed results, the Decision Tree with engineered features is now the best baseline (F1 ≈ 0.662).
- For **Accumulation Investment**, the addition of both engineered features consistently benefited all models, with the Decision Tree showing an exceptionally large gain (F1 ≈ 0.818). The engineered feature set is clearly superior for this target.

## Overall Conclusions
- **Feature Engineering Impact:**  
  The combination of Gender × Age interaction and log-transformed Income/Wealth Ratio, added to the base set, proved highly effective—especially for Accumulation Investment and particularly with the Decision Tree model. This combination yielded consistent or dramatically positive improvements for most models on the Accumulation target and boosted the top performer for the Income target.

- **Superior Feature Set:**  
  The Engineered Feature set (Base + Gender × Age + I/W Ratio) is the clear winner based on these results.

- **Best Baseline Model & Feature Set:**  
  - *Income Investment:* Decision Tree (max_depth=5) with Engineered Features (F1 ≈ 0.662)  
  - *Accumulation Investment:* Decision Tree (max_depth=5) with Engineered Features (F1 ≈ 0.818)

- **Key Insight:**  
  The Decision Tree model, which can capture interactions and non-linearities, benefited most from these added features. The dramatic improvement for Accumulation Investment suggests these features effectively isolate patterns relevant to that need.

- **Recommendation:**  
  Adopt the Engineered Feature set (Base + Gender × Age + I/W Ratio) as the primary feature set. Focus further model tuning on the Decision Tree, including exploring different `max_depth` values and regularization. Investigate feature importance to understand the drivers of this improvement.
  </body>